# RAG from Scratch: Routing
Routing is a step where we route that potentially decomposed question to the right source because it might be stored in different database or even different database types (relational db, graph db, and vector stores)

## Method 1: Logical Routing

We give the LLM knowledge of the various data sources that we have at our disposal, and we let the LLM to use its logic to reason which database to be used and apply the question to.

## Method 2: Semantic Routing

We take a question, embed it. Then we take prompts and embed it to calculate the similarity with the question.